

Here is the data exercise. As we discussed, please send it back to us by Monday. 

As part of your interview process, we would like you to complete the exercise described below where you analyze data and create a presentation.  We think this exercise should take you no more than 5-10 hours, but 
 
## Guidelines
- Your presentation can be any format you choose: PowerPoint, PDF, etc.
- We prefer that you use an open source tool like R or Python rather than a proprietary tool like SAS or Excel, but you can use whatever you like.
- All of the information you need is in the exercise.  Just be able to explain any choices you may have made about how to approach the data.
- Email your presentation and code to jayson.webb@oracle.com and ben.bendig@oracle.com when you are done.

## Exercise

**Goal**: Use the click history log to identify which combinations of Server and Domain have issues with clicks that take a long time (i.e. Big Clicks).

File: clickLog.json

Data glossary

Click duration:
chttt – Click time
chrnt – Render time

Click’s features:
wlsServer - Server
wlsDomain - Domain
 
## Things to address in your report

### Summary statistics about Click time and Render time

### The relationship between Click time and Render time

### Combinations of Server and Domain that have the biggest issues with clicks that take a long time (i.e. Big Clicks).

### Any other insights you think are useful.

